In [35]:
import os
import logging
import pickle
import osmnx as ox
import numpy as np
import geopandas as gpd
import h3
import shapely
import requests
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.ops import linemerge
from shapely import Polygon
import plotly.express as px

ox.settings.cache_folder = './data'

## inputs

In [2]:
apikey = os.getenv("GOOGLE_API_KEY")
radius = 200
step = 100
buffer = 30
categories = ['cafe', 'bakery']

In [ ]:
def h3cell_to_shapely(cell):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

In [4]:
def fetch_google_places(categories:list[str], lon:float, lat:float, radius:float, apikey:str):
    
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": apikey,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.businessStatus,places.formattedAddress,places.primaryTypeDisplayName,places.primaryType",
    }
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
            "includedTypes": categories,
            "maxResultCount": 20,
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": lat, "longitude": lon},
                    "radius": radius,
                }
            },
        }
    
    r = requests.post(url, headers=headers, json=data)
    if r.status_code != 200:
        logging.warning(f"error at request: {r.request}")
        return

    data = r.json().get('places', [])
    if not data:
        return data

    if len(data) >= 20:
        logging.warning(f'query at lon:{lon}, lat:{lat} returned 20+ results')
    
    for r in data:
        r['name'] = r['displayName']['text']
        r['geometry'] = shapely.Point(r['location']['longitude'], r['location']['latitude'])

    return data



## download network (roads)

In [ ]:
bbox = [23.7576, 37.8524, 23.7729, 37.899]
g = ox.graph_from_bbox(bbox) # we can modify filters
nodes, links = ox.graph_to_gdfs(g)

## create the linestring geometry

In [6]:
road = links[links['name'] == 'Δημητρίου Γούναρη']
crs = road.estimate_utm_crs()
geom_proj = road.to_crs(crs).union_all()
geoms_proj = gpd.GeoDataFrame(geometry=[linemerge(geom_proj)]).explode()
geoms_proj['length'] = geoms_proj.length
geom_proj = geoms_proj.sort_values(['length'], ascending=False).iloc[0].geometry
road_buffered = road.to_crs(crs).buffer(buffer).to_crs(epsg=4326).union_all()

In [ ]:
fp = './data/cafes.gpkg'
transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
length = road.to_crs(crs).length.sum()

try:
    gdf = gpd.read_file(fp)
except:
    places = []
    transformer = Transformer.from_crs(crs, "EPSG:4326")
    for distance in tqdm(np.arange(0, length, step)):
        p = shapely.line_interpolate_point(geom_proj, distance) # type:ignore
        p = transformer.transform(p.x, p.y)
        p = shapely.Point(p[1], p[0])
        
        r = fetch_google_places(categories, p.x, p.y, radius, apikey)
        if r is not None:
            places.extend(r)

    with open('./data/places.pkl', 'wb') as f:
        pickle.dump(places, f)
    
    gdf = gpd.GeoDataFrame(places, crs='epsg:4326').drop_duplicates(subset=['id'])
    gdf.to_file(fp)    

In [ ]:
cafes = gdf[gdf.intersects(road_buffered)] # keep cafes up to 30 meters from the road (inside the buffer)
cafes.head(2)

,id,formattedAddress,location,businessStatus,displayName,primaryTypeDisplayName,primaryType,name,geometry
1,ChIJgcEWYU6_oRQR-cd2bf6di68,"Attikis 1, Glifada 165 62, Greece","{'latitude': 37.894874, 'longitude': 23.761325...",OPERATIONAL,"{'text': 'coffee berry - Άνω Γλυφάδα', 'langua...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,coffee berry - Άνω Γλυφάδα,POINT (23.76133 37.89487)
2,ChIJgSMfxKu_oRQRDDje9yAhZfM,"Dim. Gounari 24, Elliniko 167 77, Greece","{'latitude': 37.893558399999996, 'longitude': ...",OPERATIONAL,"{'text': 'ROSEWOOD COFFEE HOUSE', 'languageCod...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,ROSEWOOD COFFEE HOUSE,POINT (23.76089 37.89356)


In [14]:
fig = px.scatter_map(cafes, lat=cafes.geometry.y, lon=cafes.geometry.x, text='name', hover_name="name", zoom=12)
fig.show()

In [15]:
fig = px.density_map(cafes, lat=cafes.geometry.y, lon=cafes.geometry.x, radius=12, zoom=12, map_style="open-street-map")
fig.show()

## stats / visuals

In [16]:
f'One cafe every: {length / cafes.shape[0]:.0f}m'

'One cafe every: 254m'

In [ ]:
sbbox = shapely.box(*bbox)
coords = [list(sbbox.exterior.coords)]
cells = h3.geo_to_cells(sbbox, res=9)
cells = [{'h3id': cell, 'geometry': h3cell_to_shapely(cell)} for cell in cells]
cells = gpd.GeoDataFrame(cells, crs='epsg:4326')
cells.head(2)
px.choropleth_map(cells, geojson=cells.geometry, locations=cells.index,
                  center={'lat': sbbox.centroid.y, 'lon': sbbox.centroid.x}, zoom=11.5)

,h3id,geometry
0,891eda66253ffff,"POLYGON ((23.76224 37.89638, 23.76213 37.89445..."
1,891eda67103ffff,"POLYGON ((23.75697 37.87916, 23.75686 37.87723..."


In [ ]:
counts = gpd.sjoin(cells, cafes, predicate='contains').groupby(['h3id', 'geometry'], as_index=False)['index_right'].count()
counts = counts.rename(columns={'index_right':'count'})
counts = gpd.GeoDataFrame(counts) # groupby casts to pd.DataFrame
px.choropleth_map(counts, geojson=counts.geometry, locations=counts.index,
                  color='count', color_continuous_scale=px.colors.sequential.Reds,
                  center={'lat': sbbox.centroid.y, 'lon': sbbox.centroid.x}, zoom=11.5,
                  title='Concentration of cafes (H3 cells)')